In [1]:
#hiddencell
import seaborn as sns

import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from catboost import CatBoostRegressor

fe = fm.FontEntry(fname = 'MaruBuri-Regular.otf', name = 'MaruBuri')
fm.fontManager.ttflist.insert(0, fe)
plt.rc('font', family='MaruBuri')
import pprint

In [2]:
from scipy.stats import randint
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

# **대구 교통 사고 피해 예측 AI 경진대회 Baseline Code**

## **Fixed Random Seed**  

seed 값에 의해 동일한 코드를 사용해도 결과가 다를 수 있기에, 동일한 결과를 위해 seed 값을 고정시킵니다

In [3]:
import os
import random
import numpy as np

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42)

## **데이터 불러오기 및 상위행 확인**  

train.csv, test.csv 파일을 로드하여 상위행을 출력해 봅시다

In [4]:
import pandas as pd 

train = pd.read_csv('./data/train.csv') 
test = pd.read_csv('./data/test.csv')
train_ll = pd.read_csv('./data/train+lalongtitude.csv')
test_ll = pd.read_csv('./data/test+lalongtitude.csv')

display(train.head())
display(test.head())
display(train_ll.head())
display(test_ll.head())

,ID,사고일시,요일,기상상태,시군구,도로형태,노면상태,사고유형,사고유형 - 세부분류,법규위반,...,가해운전자 상해정도,피해운전자 차종,피해운전자 성별,피해운전자 연령,피해운전자 상해정도,사망자수,중상자수,경상자수,부상자수,ECLO
0,ACCIDENT_00000,2019-01-01 00,화요일,맑음,대구광역시 중구 대신동,단일로 - 기타,건조,차대사람,길가장자리구역통행중,안전운전불이행,...,상해없음,보행자,여,70세,중상,0,1,0,0,5
1,ACCIDENT_00001,2019-01-01 00,화요일,흐림,대구광역시 달서구 감삼동,단일로 - 기타,건조,차대사람,보도통행중,기타,...,상해없음,보행자,남,61세,경상,0,0,1,0,3
2,ACCIDENT_00002,2019-01-01 01,화요일,맑음,대구광역시 수성구 두산동,단일로 - 기타,건조,차대사람,차도통행중,안전운전불이행,...,상해없음,보행자,남,38세,경상,0,0,1,0,3
3,ACCIDENT_00003,2019-01-01 02,화요일,맑음,대구광역시 북구 복현동,단일로 - 기타,건조,차대차,추돌,안전운전불이행,...,상해없음,승용,남,36세,중상,0,1,0,0,5
4,ACCIDENT_00004,2019-01-01 04,화요일,맑음,대구광역시 동구 신암동,단일로 - 기타,건조,차대차,추돌,안전운전불이행,...,상해없음,승용,남,52세,경상,0,0,1,0,3


,ID,사고일시,요일,기상상태,시군구,도로형태,노면상태,사고유형
0,ACCIDENT_39609,2022-01-01 01,토요일,맑음,대구광역시 수성구 상동,교차로 - 교차로안,건조,차대사람
1,ACCIDENT_39610,2022-01-01 01,토요일,맑음,대구광역시 수성구 지산동,단일로 - 기타,건조,차대사람
2,ACCIDENT_39611,2022-01-01 04,토요일,맑음,대구광역시 수성구 수성동2가,교차로 - 교차로안,건조,차대차
3,ACCIDENT_39612,2022-01-01 04,토요일,맑음,대구광역시 수성구 신매동,단일로 - 기타,건조,차대차
4,ACCIDENT_39613,2022-01-01 06,토요일,맑음,대구광역시 달서구 감삼동,교차로 - 교차로안,건조,차대차


,도시,구,동,위도,경도
0,대구광역시,중구,대신동,35.854356,128.613815
1,대구광역시,달서구,감삼동,35.842321,128.538606
2,대구광역시,수성구,두산동,35.869404,128.620835
3,대구광역시,북구,복현동,35.869404,128.620835
4,대구광역시,동구,신암동,35.870662,128.594995


,도시,구,동,위도,경도
0,대구광역시,수성구,상동,35.869404,128.620835
1,대구광역시,수성구,지산동,35.868951,128.594995
2,대구광역시,수성구,수성동2가,35.869404,128.620835
3,대구광역시,수성구,신매동,35.870662,128.594995
4,대구광역시,달서구,감삼동,35.842321,128.538606


## 위도, 경도 컬럼 train 데이터와 test 데이터에 추가하기

In [5]:
train['위도'] = train_ll['위도']
test['위도'] = test_ll['위도']
train['경도'] = train_ll['경도']
test['경도'] = test_ll['경도']

In [6]:
display(train.head())
display(test.head())

,ID,사고일시,요일,기상상태,시군구,도로형태,노면상태,사고유형,사고유형 - 세부분류,법규위반,...,피해운전자 성별,피해운전자 연령,피해운전자 상해정도,사망자수,중상자수,경상자수,부상자수,ECLO,위도,경도
0,ACCIDENT_00000,2019-01-01 00,화요일,맑음,대구광역시 중구 대신동,단일로 - 기타,건조,차대사람,길가장자리구역통행중,안전운전불이행,...,여,70세,중상,0,1,0,0,5,35.854356,128.613815
1,ACCIDENT_00001,2019-01-01 00,화요일,흐림,대구광역시 달서구 감삼동,단일로 - 기타,건조,차대사람,보도통행중,기타,...,남,61세,경상,0,0,1,0,3,35.842321,128.538606
2,ACCIDENT_00002,2019-01-01 01,화요일,맑음,대구광역시 수성구 두산동,단일로 - 기타,건조,차대사람,차도통행중,안전운전불이행,...,남,38세,경상,0,0,1,0,3,35.869404,128.620835
3,ACCIDENT_00003,2019-01-01 02,화요일,맑음,대구광역시 북구 복현동,단일로 - 기타,건조,차대차,추돌,안전운전불이행,...,남,36세,중상,0,1,0,0,5,35.869404,128.620835
4,ACCIDENT_00004,2019-01-01 04,화요일,맑음,대구광역시 동구 신암동,단일로 - 기타,건조,차대차,추돌,안전운전불이행,...,남,52세,경상,0,0,1,0,3,35.870662,128.594995


,ID,사고일시,요일,기상상태,시군구,도로형태,노면상태,사고유형,위도,경도
0,ACCIDENT_39609,2022-01-01 01,토요일,맑음,대구광역시 수성구 상동,교차로 - 교차로안,건조,차대사람,35.869404,128.620835
1,ACCIDENT_39610,2022-01-01 01,토요일,맑음,대구광역시 수성구 지산동,단일로 - 기타,건조,차대사람,35.868951,128.594995
2,ACCIDENT_39611,2022-01-01 04,토요일,맑음,대구광역시 수성구 수성동2가,교차로 - 교차로안,건조,차대차,35.869404,128.620835
3,ACCIDENT_39612,2022-01-01 04,토요일,맑음,대구광역시 수성구 신매동,단일로 - 기타,건조,차대차,35.870662,128.594995
4,ACCIDENT_39613,2022-01-01 06,토요일,맑음,대구광역시 달서구 감삼동,교차로 - 교차로안,건조,차대차,35.842321,128.538606


## **train, test 데이터 기간 확인하기**  

학습(train) 데이터의 기간과 예측 대상이 되는 test 데이터의 기간을 살펴 봅니다

In [7]:
display(f"train : {train.iloc[0]['사고일시']} ~ {train.iloc[-1]['사고일시']}")
display(f"test : {test.iloc[0]['사고일시']} ~ {test.iloc[-1]['사고일시']}")     

'train : 2019-01-01 00 ~ 2021-12-31 23'

'test : 2022-01-01 01 ~ 2022-12-31 21'

## **예측 대상 확인하기**

sample_submission.csv를 로드하여 출력 형식을 확인해 봄으로서, 
대회 페이지 규칙에 기술되어 있는, 예측 대상이 무엇인지 확인해 봅시다.

최종적으로 예측해야 하는 값은 'ECLO'(인명피해 심각도) 값입니다. 

그러나, train데이터 컬럼에 있는 '사망자수', '중상자수', '경상자수', '부상자수'를 각각 예측하여 ECLO 예측값을 구할 수도 있습니다. 즉, ECLO 값을 예측하는 단일 모델로 접근할 수도,  각각을 별도로 예측하는 모델로 접근할 수도 있습니다. 

<img src="https://drive.google.com/uc?id=19PvSqu8k04D3a-Bskcl-d-Xkckkn2Fvv">  

In [8]:
sample_submission = pd.read_csv("./data/sample_submission.csv")
sample_submission.head()

,ID,ECLO
0,ACCIDENT_39609,0
1,ACCIDENT_39610,0
2,ACCIDENT_39611,0
3,ACCIDENT_39612,0
4,ACCIDENT_39613,0


## **데이터 전처리**  

현재 '사고일시', '시군구', '도로형태' 컬럼은 반복되는 패턴으로 여러 정보를 포함하고 있습니다
이런 반복되는 패턴을 일반화하면 pandas에서 제공하는 str.extract를 통해 한 번에 추출 가능합니다  

## **파생 변수 생성 1 : 날짜, 시간정보 생성**

'사고일시' 컬럼으로 부터 연도, 월, 일, 시간 정보 추출 및 변환 합니다

In [9]:
train_df = train.copy()
test_df = test.copy()

time_pattern = r'(\d{4})-(\d{1,2})-(\d{1,2}) (\d{1,2})' 

train_df[['연', '월', '일', '시간']] = train['사고일시'].str.extract(time_pattern)
train_df[['연', '월', '일', '시간']] = train_df[['연', '월', '일', '시간']].apply(pd.to_numeric) # 추출된 문자열을 수치화해줍니다 
train_df = train_df.drop(columns=['사고일시']) # 정보 추출이 완료된 '사고일시' 컬럼은 제거합니다 

# 해당 과정을 test_x에 대해서도 반복해줍니다 
test_df[['연', '월', '일', '시간']] = test['사고일시'].str.extract(time_pattern)
test_df[['연', '월', '일', '시간']] = test_df[['연', '월', '일', '시간']].apply(pd.to_numeric)
test_df = test_df.drop(columns=['사고일시'])

# display(f"columns of train_df : {train_df.columns}")
# display(f"columns of test_df : {test_df.columns}")

## **파생 변수 생성 2 : 공간(위치) 정보 생성** 

'시군구' 컬럼으로부터 의미 있는 공산 정보를 추출 및 변환 합니다

In [10]:
location_pattern = r'(\S+) (\S+) (\S+)'

train_df[['도시', '구', '동']] = train['시군구'].str.extract(location_pattern)
train_df = train_df.drop(columns=['시군구'])

test_df[['도시', '구', '동']] = test['시군구'].str.extract(location_pattern)
test_df = test_df.drop(columns=['시군구'])

# display(f"columns of train_df : {train_df.columns}")
# display(f"columns of test_df : {test_df.columns}")

## **파생 변수 추출 3 : 도로 형태 정보 추출**  

'도로형태' 컬럼은 '단일로 - 기타'와 같은 패턴으로 구성되어 있습니다. 이를 두종류의 독립된 정보로 보고 두개의 컬럼으로 분리하여 생성합니다.

In [11]:
road_pattern = r'(.+) - (.+)'

train_df[['도로형태1', '도로형태2']] = train['도로형태'].str.extract(road_pattern)
train_df = train_df.drop(columns=['도로형태'])

test_df[['도로형태1', '도로형태2']] = test['도로형태'].str.extract(road_pattern)
test_df = test_df.drop(columns=['도로형태'])

# display(f"columns of train_df : {train_df.columns}")
# display(f"columns of test_df : {test_df.columns}")

## **전처리 결과 확인**

지금까지 전처리한 결과를 확인해 봅시다 

In [12]:
display(train_df.head())
display(test_df.head())

,ID,요일,기상상태,노면상태,사고유형,사고유형 - 세부분류,법규위반,가해운전자 차종,가해운전자 성별,가해운전자 연령,...,경도,연,월,일,시간,도시,구,동,도로형태1,도로형태2
0,ACCIDENT_00000,화요일,맑음,건조,차대사람,길가장자리구역통행중,안전운전불이행,승용,여,51세,...,128.613815,2019,1,1,0,대구광역시,중구,대신동,단일로,기타
1,ACCIDENT_00001,화요일,흐림,건조,차대사람,보도통행중,기타,승용,남,39세,...,128.538606,2019,1,1,0,대구광역시,달서구,감삼동,단일로,기타
2,ACCIDENT_00002,화요일,맑음,건조,차대사람,차도통행중,안전운전불이행,승용,남,70세,...,128.620835,2019,1,1,1,대구광역시,수성구,두산동,단일로,기타
3,ACCIDENT_00003,화요일,맑음,건조,차대차,추돌,안전운전불이행,승용,남,49세,...,128.620835,2019,1,1,2,대구광역시,북구,복현동,단일로,기타
4,ACCIDENT_00004,화요일,맑음,건조,차대차,추돌,안전운전불이행,승용,남,30세,...,128.594995,2019,1,1,4,대구광역시,동구,신암동,단일로,기타


,ID,요일,기상상태,노면상태,사고유형,위도,경도,연,월,일,시간,도시,구,동,도로형태1,도로형태2
0,ACCIDENT_39609,토요일,맑음,건조,차대사람,35.869404,128.620835,2022,1,1,1,대구광역시,수성구,상동,교차로,교차로안
1,ACCIDENT_39610,토요일,맑음,건조,차대사람,35.868951,128.594995,2022,1,1,1,대구광역시,수성구,지산동,단일로,기타
2,ACCIDENT_39611,토요일,맑음,건조,차대차,35.869404,128.620835,2022,1,1,4,대구광역시,수성구,수성동2가,교차로,교차로안
3,ACCIDENT_39612,토요일,맑음,건조,차대차,35.870662,128.594995,2022,1,1,4,대구광역시,수성구,신매동,단일로,기타
4,ACCIDENT_39613,토요일,맑음,건조,차대차,35.842321,128.538606,2022,1,1,6,대구광역시,달서구,감삼동,교차로,교차로안


### 지금까지 전처리한 train, test 데이터 저장

In [13]:
# _latitude_longtitude/preprocessing / 위도경도 합쳤고, 파생 변수 3개 생성한 데이터
train_df.to_csv('./data/trainll_pp.csv', index = False)
test_df.to_csv('./data/testll_pp.csv', index = False)

## **train, test 데이터의 독립변수, 종속 변수 분리**

모델 추론 시점에서 획득할 수 있는 데이터는 학습 시점에서 활용할 수 있는 데이터와 다를 수 있습니다  
그렇기에 본 baseline에서는 test 시점에서만 활용할 수 있는 데이터들(test의 column들)만 사용하여 train 데이터를 구성했습니다 

train_df 중에서 test_x가 가진 column들만 추출하여 train_x(독립변수)를 구성하고, train_df 중에서 예측 목표인 EPDO를 추출하여 train_y(종속변수)를 구성합니다

In [14]:
test_x = test_df.drop(columns=['ID']).copy()
train_x = train_df[test_x.columns].copy()
train_y = train_df['ECLO'].copy()

In [15]:
display(test_x.head())
display(train_x.head())

,요일,기상상태,노면상태,사고유형,위도,경도,연,월,일,시간,도시,구,동,도로형태1,도로형태2
0,토요일,맑음,건조,차대사람,35.869404,128.620835,2022,1,1,1,대구광역시,수성구,상동,교차로,교차로안
1,토요일,맑음,건조,차대사람,35.868951,128.594995,2022,1,1,1,대구광역시,수성구,지산동,단일로,기타
2,토요일,맑음,건조,차대차,35.869404,128.620835,2022,1,1,4,대구광역시,수성구,수성동2가,교차로,교차로안
3,토요일,맑음,건조,차대차,35.870662,128.594995,2022,1,1,4,대구광역시,수성구,신매동,단일로,기타
4,토요일,맑음,건조,차대차,35.842321,128.538606,2022,1,1,6,대구광역시,달서구,감삼동,교차로,교차로안


,요일,기상상태,노면상태,사고유형,위도,경도,연,월,일,시간,도시,구,동,도로형태1,도로형태2
0,화요일,맑음,건조,차대사람,35.854356,128.613815,2019,1,1,0,대구광역시,중구,대신동,단일로,기타
1,화요일,흐림,건조,차대사람,35.842321,128.538606,2019,1,1,0,대구광역시,달서구,감삼동,단일로,기타
2,화요일,맑음,건조,차대사람,35.869404,128.620835,2019,1,1,1,대구광역시,수성구,두산동,단일로,기타
3,화요일,맑음,건조,차대차,35.869404,128.620835,2019,1,1,2,대구광역시,북구,복현동,단일로,기타
4,화요일,맑음,건조,차대차,35.870662,128.594995,2019,1,1,4,대구광역시,동구,신암동,단일로,기타


## **범주형(Categorical) 변수, 수치형 변수로 변환하기**

모델 학습을 위해 train_x의 문자열 타입의 컬럼들을 추출하고, LabelEncoder를 활용하여 이 컬럼들을 모두 수치형 변수로 변환해 보겠습니다

In [16]:
from sklearn.preprocessing import LabelEncoder

categorical_features = list(train_x.dtypes[train_x.dtypes == "object"].index)
# 추출된 문자열 변수 확인
display(categorical_features)

for i in categorical_features:
    le = LabelEncoder()
    le=le.fit(train_x[i]) 
    train_x[i]=le.transform(train_x[i])
    
    for case in np.unique(test_x[i]):
        if case not in le.classes_: 
            le.classes_ = np.append(le.classes_, case) 
    test_x[i]=le.transform(test_x[i])
    
display(train_x.head())
display(test_x.head())

['요일', '기상상태', '노면상태', '사고유형', '도시', '구', '동', '도로형태1', '도로형태2']

,요일,기상상태,노면상태,사고유형,위도,경도,연,월,일,시간,도시,구,동,도로형태1,도로형태2
0,6,2,0,0,35.854356,128.613815,2019,1,1,0,0,7,40,2,5
1,6,5,0,0,35.842321,128.538606,2019,1,1,0,0,1,4,2,5
2,6,2,0,0,35.869404,128.620835,2019,1,1,1,0,6,66,2,5
3,6,2,0,1,35.869404,128.620835,2019,1,1,2,0,4,79,2,5
4,6,2,0,1,35.870662,128.594995,2019,1,1,4,0,3,129,2,5


,요일,기상상태,노면상태,사고유형,위도,경도,연,월,일,시간,도시,구,동,도로형태1,도로형태2
0,5,2,0,0,35.869404,128.620835,2022,1,1,1,0,6,99,0,3
1,5,2,0,0,35.868951,128.594995,2022,1,1,1,0,6,168,2,5
2,5,2,0,1,35.869404,128.620835,2022,1,1,4,0,6,117,0,3
3,5,2,0,1,35.870662,128.594995,2022,1,1,4,0,6,126,2,5
4,5,2,0,1,35.842321,128.538606,2022,1,1,6,0,1,4,0,3


## **Model Train & Prediction** 

전체 학습 데이터에 대하여 학습후 최종 예측을 합니다. 여기서는 DecisionTreeRegressor 모델로 학습하고 예측하는 예시 코드입니다.

In [17]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
import catboost
from scipy.stats import randint, loguniform
import time

# 데이터를 학습용과 테스트용으로 나누기
Train_x, Test_x, Train_y, Test_y = train_test_split(train_x, train_y, 
                                                    test_size=0.3, 
                                                    random_state=37)

param_distribs = {
    'n_estimators': randint(100, 300),
    'depth': randint(1, 5),
    'learning_rate': loguniform(1e-3, 0.1),
    'min_child_samples': randint(10, 40),
    'grow_policy': ['SymmetricTree', 'Lossguide', 'Depthwise']
}

start = time.time()

# CatBoost 모델 초기화
modelcat = catboost.CatBoostRegressor(eval_metric='R2')

# 랜덤 서치를 통한 하이퍼파라미터 튜닝
modelcat.randomized_search(param_distribs,
                              Train_x, Train_y,
                              cv=3, n_iter=30)

# 있어도 되고 없어도 됨. 위에서 학습이 됨.
#modelcat_la.fit(Train_x, Train_y)

# 학습된 모델을 사용하여 예측 학습데이터에서 분리되어 나온 test 데이터로 하는 것.
prediction = modelcat.predict(Test_x)

# R2 계수
r2 = r2_score(Test_y, prediction)

# 평균제곱오차(MSE)
mse = mean_squared_error(Test_y, prediction)

preds = modelcat.predict(test_x)

end = time.time()
print(f'Time elapsed: {end - start}')


print(f'R2 score on training set: {r2}')
print(f'Mean Squared Error on test set: {mse}')

0:	learn: -2.1548798	test: -2.1808892	best: -2.1808892 (0)	total: 55.1ms	remaining: 10.5s
1:	learn: -2.1448628	test: -2.1708120	best: -2.1708120 (1)	total: 58.3ms	remaining: 5.54s
2:	learn: -2.1348314	test: -2.1607177	best: -2.1607177 (2)	total: 62.3ms	remaining: 3.92s
3:	learn: -2.1249157	test: -2.1507430	best: -2.1507430 (3)	total: 64.2ms	remaining: 3.02s
4:	learn: -2.1150404	test: -2.1408178	best: -2.1408178 (4)	total: 69.8ms	remaining: 2.61s
5:	learn: -2.1052089	test: -2.1309342	best: -2.1309342 (5)	total: 73.2ms	remaining: 2.27s
6:	learn: -2.0954319	test: -2.1211113	best: -2.1211113 (6)	total: 78.3ms	remaining: 2.07s
7:	learn: -2.0857002	test: -2.1113208	best: -2.1113208 (7)	total: 84.2ms	remaining: 1.94s
8:	learn: -2.0759978	test: -2.1015630	best: -2.1015630 (8)	total: 87.7ms	remaining: 1.78s
9:	learn: -2.0663554	test: -2.0918695	best: -2.0918695 (9)	total: 90.9ms	remaining: 1.65s
10:	learn: -2.0567592	test: -2.0822166	best: -2.0822166 (10)	total: 95ms	remaining: 1.56s
11:	learn:

128:	learn: -1.1838998	test: -1.2031420	best: -1.2031420 (128)	total: 661ms	remaining: 323ms
129:	learn: -1.1783702	test: -1.1975655	best: -1.1975655 (129)	total: 666ms	remaining: 318ms
130:	learn: -1.1728765	test: -1.1920275	best: -1.1920275 (130)	total: 674ms	remaining: 314ms
131:	learn: -1.1674059	test: -1.1865191	best: -1.1865191 (131)	total: 680ms	remaining: 309ms
132:	learn: -1.1619525	test: -1.1810275	best: -1.1810275 (132)	total: 687ms	remaining: 305ms
133:	learn: -1.1565384	test: -1.1755745	best: -1.1755745 (133)	total: 691ms	remaining: 299ms
134:	learn: -1.1510895	test: -1.1700743	best: -1.1700743 (134)	total: 697ms	remaining: 294ms
135:	learn: -1.1456584	test: -1.1645854	best: -1.1645854 (135)	total: 704ms	remaining: 290ms
136:	learn: -1.1402598	test: -1.1591343	best: -1.1591343 (136)	total: 708ms	remaining: 284ms
137:	learn: -1.1349419	test: -1.1537762	best: -1.1537762 (137)	total: 711ms	remaining: 278ms
138:	learn: -1.1296517	test: -1.1484415	best: -1.1484415 (138)	total: 

67:	learn: -1.6368337	test: -1.6595782	best: -1.6595782 (67)	total: 135ms	remaining: 103ms
68:	learn: -1.6301232	test: -1.6528222	best: -1.6528222 (68)	total: 138ms	remaining: 102ms
69:	learn: -1.6234402	test: -1.6460946	best: -1.6460946 (69)	total: 141ms	remaining: 101ms
70:	learn: -1.6167845	test: -1.6393947	best: -1.6393947 (70)	total: 145ms	remaining: 100ms
71:	learn: -1.6101569	test: -1.6327240	best: -1.6327240 (71)	total: 147ms	remaining: 98.1ms
72:	learn: -1.6035540	test: -1.6260785	best: -1.6260785 (72)	total: 150ms	remaining: 96.8ms
73:	learn: -1.5969806	test: -1.6194616	best: -1.6194616 (73)	total: 153ms	remaining: 95.3ms
74:	learn: -1.5904340	test: -1.6128725	best: -1.6128725 (74)	total: 157ms	remaining: 94ms
75:	learn: -1.5839130	test: -1.6063103	best: -1.6063103 (75)	total: 158ms	remaining: 91.6ms
76:	learn: -1.5773584	test: -1.5997000	best: -1.5997000 (76)	total: 160ms	remaining: 89.2ms
77:	learn: -1.5708914	test: -1.5931890	best: -1.5931890 (77)	total: 161ms	remaining: 8

85:	learn: -0.5111590	test: -0.5234271	best: -0.5234271 (85)	total: 263ms	remaining: 269ms
86:	learn: -0.5024261	test: -0.5145612	best: -0.5145612 (86)	total: 266ms	remaining: 266ms
87:	learn: -0.4940079	test: -0.5060461	best: -0.5060461 (87)	total: 272ms	remaining: 265ms
88:	learn: -0.4857131	test: -0.4976562	best: -0.4976562 (88)	total: 275ms	remaining: 263ms
89:	learn: -0.4773956	test: -0.4892161	best: -0.4892161 (89)	total: 279ms	remaining: 260ms
90:	learn: -0.4693908	test: -0.4811197	best: -0.4811197 (90)	total: 283ms	remaining: 258ms
91:	learn: -0.4613275	test: -0.4729370	best: -0.4729370 (91)	total: 286ms	remaining: 255ms
92:	learn: -0.4535897	test: -0.4650930	best: -0.4650930 (92)	total: 291ms	remaining: 254ms
93:	learn: -0.4458159	test: -0.4571922	best: -0.4571922 (93)	total: 294ms	remaining: 251ms
94:	learn: -0.4381759	test: -0.4494165	best: -0.4494165 (94)	total: 296ms	remaining: 246ms
95:	learn: -0.4306566	test: -0.4417700	best: -0.4417700 (95)	total: 299ms	remaining: 243ms

35:	learn: -0.3140202	test: -0.3238068	best: -0.3238068 (35)	total: 121ms	remaining: 562ms
36:	learn: -0.2972134	test: -0.3067157	best: -0.3067157 (36)	total: 126ms	remaining: 563ms
37:	learn: -0.2808990	test: -0.2901099	best: -0.2901099 (37)	total: 131ms	remaining: 567ms
38:	learn: -0.2654572	test: -0.2743573	best: -0.2743573 (38)	total: 136ms	remaining: 570ms
39:	learn: -0.2508165	test: -0.2593896	best: -0.2593896 (39)	total: 142ms	remaining: 578ms
40:	learn: -0.2372574	test: -0.2455965	best: -0.2455965 (40)	total: 147ms	remaining: 580ms
41:	learn: -0.2240138	test: -0.2320280	best: -0.2320280 (41)	total: 151ms	remaining: 578ms
42:	learn: -0.2115002	test: -0.2192095	best: -0.2192095 (42)	total: 155ms	remaining: 577ms
43:	learn: -0.1996444	test: -0.2070492	best: -0.2070492 (43)	total: 159ms	remaining: 576ms
44:	learn: -0.1883498	test: -0.1955368	best: -0.1955368 (44)	total: 163ms	remaining: 573ms
45:	learn: -0.1776726	test: -0.1845808	best: -0.1845808 (45)	total: 166ms	remaining: 565ms

...

104:	learn: 0.0306230	test: 0.0240205	best: 0.0240205 (104)	total: 833ms	remaining: 1.25s
105:	learn: 0.0307860	test: 0.0241167	best: 0.0241167 (105)	total: 845ms	remaining: 1.24s
106:	learn: 0.0310109	test: 0.0242110	best: 0.0242110 (106)	total: 856ms	remaining: 1.24s
107:	learn: 0.0312163	test: 0.0243750	best: 0.0243750 (107)	total: 871ms	remaining: 1.24s
108:	learn: 0.0313940	test: 0.0244679	best: 0.0244679 (108)	total: 879ms	remaining: 1.23s
109:	learn: 0.0316590	test: 0.0246418	best: 0.0246418 (109)	total: 884ms	remaining: 1.22s
110:	learn: 0.0319886	test: 0.0248280	best: 0.0248280 (110)	total: 888ms	remaining: 1.21s
111:	learn: 0.0323089	test: 0.0249309	best: 0.0249309 (111)	total: 895ms	remaining: 1.2s
112:	learn: 0.0325552	test: 0.0249908	best: 0.0249908 (112)	total: 904ms	remaining: 1.19s
113:	learn: 0.0327220	test: 0.0251033	best: 0.0251033 (113)	total: 912ms	remaining: 1.18s
114:	learn: 0.0328607	test: 0.0251818	best: 0.0251818 (114)	total: 917ms	remaining: 1.17s
115:	learn:

200:	learn: 0.0449835	test: 0.0288571	best: 0.0288706 (199)	total: 1.65s	remaining: 502ms
201:	learn: 0.0451101	test: 0.0289390	best: 0.0289390 (201)	total: 1.67s	remaining: 496ms
202:	learn: 0.0452011	test: 0.0289712	best: 0.0289712 (202)	total: 1.68s	remaining: 489ms
203:	learn: 0.0453487	test: 0.0289845	best: 0.0289845 (203)	total: 1.69s	remaining: 482ms
204:	learn: 0.0453909	test: 0.0289992	best: 0.0289992 (204)	total: 1.7s	remaining: 473ms
205:	learn: 0.0455185	test: 0.0289716	best: 0.0289992 (204)	total: 1.71s	remaining: 464ms
206:	learn: 0.0456256	test: 0.0289645	best: 0.0289992 (204)	total: 1.72s	remaining: 456ms
207:	learn: 0.0457051	test: 0.0289483	best: 0.0289992 (204)	total: 1.72s	remaining: 447ms
208:	learn: 0.0458089	test: 0.0289471	best: 0.0289992 (204)	total: 1.73s	remaining: 440ms
209:	learn: 0.0458536	test: 0.0289978	best: 0.0289992 (204)	total: 1.74s	remaining: 431ms
210:	learn: 0.0459125	test: 0.0289853	best: 0.0289992 (204)	total: 1.75s	remaining: 422ms
211:	learn:

43:	learn: -0.0548488	test: -0.0608966	best: -0.0608966 (43)	total: 434ms	remaining: 2.15s
44:	learn: -0.0492191	test: -0.0551415	best: -0.0551415 (44)	total: 451ms	remaining: 2.17s
45:	learn: -0.0441056	test: -0.0499679	best: -0.0499679 (45)	total: 467ms	remaining: 2.19s
46:	learn: -0.0393438	test: -0.0451386	best: -0.0451386 (46)	total: 481ms	remaining: 2.2s
47:	learn: -0.0347728	test: -0.0404357	best: -0.0404357 (47)	total: 490ms	remaining: 2.19s
48:	learn: -0.0305088	test: -0.0360524	best: -0.0360524 (48)	total: 501ms	remaining: 2.18s
49:	learn: -0.0265744	test: -0.0319970	best: -0.0319970 (49)	total: 514ms	remaining: 2.18s
50:	learn: -0.0230748	test: -0.0284491	best: -0.0284491 (50)	total: 526ms	remaining: 2.17s
51:	learn: -0.0195904	test: -0.0249293	best: -0.0249293 (51)	total: 541ms	remaining: 2.19s
52:	learn: -0.0164515	test: -0.0216726	best: -0.0216726 (52)	total: 554ms	remaining: 2.18s
53:	learn: -0.0134192	test: -0.0185924	best: -0.0185924 (53)	total: 567ms	remaining: 2.18s


146:	learn: 0.0374318	test: 0.0273840	best: 0.0273840 (146)	total: 1.48s	remaining: 1.16s
147:	learn: 0.0375795	test: 0.0275031	best: 0.0275031 (147)	total: 1.49s	remaining: 1.15s
148:	learn: 0.0376855	test: 0.0275775	best: 0.0275775 (148)	total: 1.5s	remaining: 1.14s
149:	learn: 0.0378131	test: 0.0276693	best: 0.0276693 (149)	total: 1.51s	remaining: 1.13s
150:	learn: 0.0379382	test: 0.0277677	best: 0.0277677 (150)	total: 1.51s	remaining: 1.11s
151:	learn: 0.0381572	test: 0.0278117	best: 0.0278117 (151)	total: 1.52s	remaining: 1.1s
152:	learn: 0.0383585	test: 0.0278642	best: 0.0278642 (152)	total: 1.53s	remaining: 1.09s
153:	learn: 0.0385410	test: 0.0279041	best: 0.0279041 (153)	total: 1.53s	remaining: 1.08s
154:	learn: 0.0386369	test: 0.0279295	best: 0.0279295 (154)	total: 1.54s	remaining: 1.06s
155:	learn: 0.0388102	test: 0.0280093	best: 0.0280093 (155)	total: 1.54s	remaining: 1.05s
156:	learn: 0.0389029	test: 0.0280440	best: 0.0280440 (156)	total: 1.55s	remaining: 1.04s
157:	learn: 

254:	learn: 0.0511082	test: 0.0296068	best: 0.0297094 (240)	total: 2.28s	remaining: 62.7ms
255:	learn: 0.0513963	test: 0.0294952	best: 0.0297094 (240)	total: 2.3s	remaining: 53.8ms
256:	learn: 0.0515621	test: 0.0294907	best: 0.0297094 (240)	total: 2.31s	remaining: 44.9ms
257:	learn: 0.0516791	test: 0.0294608	best: 0.0297094 (240)	total: 2.32s	remaining: 36ms
258:	learn: 0.0518213	test: 0.0295068	best: 0.0297094 (240)	total: 2.33s	remaining: 27ms
259:	learn: 0.0519267	test: 0.0295063	best: 0.0297094 (240)	total: 2.33s	remaining: 17.9ms
260:	learn: 0.0520879	test: 0.0295071	best: 0.0297094 (240)	total: 2.34s	remaining: 8.96ms
261:	learn: 0.0521808	test: 0.0295371	best: 0.0297094 (240)	total: 2.35s	remaining: 0us

bestTest = 0.02970940911
bestIteration = 240

Time elapsed: 34.15486001968384
R2 score on training set: 0.034571194458440924
Mean Squared Error on test set: 10.015146701410591


In [20]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
import catboost
from scipy.stats import randint, loguniform
import time

# # 데이터를 학습용과 테스트용으로 나누기
# Train_x, Test_x, Train_y, Test_y = train_test_split(train_x, train_y, 
#                                                     test_size=0.3, 
#                                                     random_state=37)

param_distribs = {
    'n_estimators': randint(100, 300),
    'depth': randint(1, 5),
    'learning_rate': loguniform(1e-3, 0.1),
    'min_child_samples': randint(10, 40),
    'grow_policy': ['SymmetricTree', 'Lossguide', 'Depthwise']
}

start = time.time()

# CatBoost 모델 초기화
modelcat2 = catboost.CatBoostRegressor(eval_metric='R2')

# 랜덤 서치를 통한 하이퍼파라미터 튜닝
modelcat2.randomized_search(param_distribs,
                              train_x, train_y,
                              cv=3, n_iter=30)

# 있어도 되고 없어도 됨. 위에서 학습이 됨.
#modelcat_la.fit(Train_x, Train_y)

# 학습된 모델을 사용하여 예측 학습데이터에서 분리되어 나온 test 데이터로 하는 것.
preds2 = modelcat2.predict(test_x)

end = time.time()
print(f'Time elapsed: {end - start}')

0:	learn: -2.0711607	test: -2.1087095	best: -2.1087095 (0)	total: 6.1ms	remaining: 1.52s
1:	learn: -1.9820146	test: -2.0182800	best: -2.0182800 (1)	total: 12.3ms	remaining: 1.53s
2:	learn: -1.8967080	test: -1.9317022	best: -1.9317022 (2)	total: 15ms	remaining: 1.23s
3:	learn: -1.8150631	test: -1.8488469	best: -1.8488469 (3)	total: 18.1ms	remaining: 1.11s
4:	learn: -1.7368285	test: -1.7694860	best: -1.7694860 (4)	total: 20.4ms	remaining: 1s
5:	learn: -1.6620340	test: -1.6935367	best: -1.6935367 (5)	total: 22.6ms	remaining: 918ms
6:	learn: -1.5897218	test: -1.6201569	best: -1.6201569 (6)	total: 25.4ms	remaining: 883ms
7:	learn: -1.5211959	test: -1.5506287	best: -1.5506287 (7)	total: 30.1ms	remaining: 909ms
8:	learn: -1.4550082	test: -1.4834960	best: -1.4834960 (8)	total: 34.9ms	remaining: 934ms
9:	learn: -1.3923111	test: -1.4198751	best: -1.4198751 (9)	total: 39.2ms	remaining: 942ms
10:	learn: -1.3323116	test: -1.3589685	best: -1.3589685 (10)	total: 44.6ms	remaining: 970ms
11:	learn: -1.

110:	learn: 0.0030616	test: 0.0024753	best: 0.0024753 (110)	total: 409ms	remaining: 512ms
111:	learn: 0.0038885	test: 0.0033836	best: 0.0033836 (111)	total: 416ms	remaining: 512ms
112:	learn: 0.0046157	test: 0.0041411	best: 0.0041411 (112)	total: 423ms	remaining: 513ms
113:	learn: 0.0053124	test: 0.0048799	best: 0.0048799 (113)	total: 431ms	remaining: 514ms
114:	learn: 0.0060222	test: 0.0056179	best: 0.0056179 (114)	total: 442ms	remaining: 519ms
115:	learn: 0.0067459	test: 0.0063712	best: 0.0063712 (115)	total: 450ms	remaining: 520ms
116:	learn: 0.0074421	test: 0.0070970	best: 0.0070970 (116)	total: 459ms	remaining: 522ms
117:	learn: 0.0080303	test: 0.0077005	best: 0.0077005 (117)	total: 467ms	remaining: 522ms
118:	learn: 0.0086186	test: 0.0083380	best: 0.0083380 (118)	total: 474ms	remaining: 522ms
119:	learn: 0.0091782	test: 0.0089379	best: 0.0089379 (119)	total: 479ms	remaining: 519ms
120:	learn: 0.0097566	test: 0.0095679	best: 0.0095679 (120)	total: 482ms	remaining: 513ms
121:	learn

227:	learn: 0.0262045	test: 0.0278487	best: 0.0278487 (227)	total: 1.01s	remaining: 97.6ms
228:	learn: 0.0262653	test: 0.0279303	best: 0.0279303 (228)	total: 1.02s	remaining: 93.3ms
229:	learn: 0.0263153	test: 0.0279581	best: 0.0279581 (229)	total: 1.02s	remaining: 88.8ms
230:	learn: 0.0263536	test: 0.0279703	best: 0.0279703 (230)	total: 1.03s	remaining: 84.5ms
231:	learn: 0.0263851	test: 0.0279812	best: 0.0279812 (231)	total: 1.03s	remaining: 80.1ms
232:	learn: 0.0264540	test: 0.0280500	best: 0.0280500 (232)	total: 1.04s	remaining: 75.7ms
233:	learn: 0.0264854	test: 0.0281029	best: 0.0281029 (233)	total: 1.04s	remaining: 71.1ms
234:	learn: 0.0265297	test: 0.0281984	best: 0.0281984 (234)	total: 1.05s	remaining: 66.8ms
235:	learn: 0.0265891	test: 0.0282647	best: 0.0282647 (235)	total: 1.05s	remaining: 62.4ms
236:	learn: 0.0266294	test: 0.0282752	best: 0.0282752 (236)	total: 1.06s	remaining: 57.9ms
237:	learn: 0.0266599	test: 0.0282866	best: 0.0282866 (237)	total: 1.06s	remaining: 53.5ms

82:	learn: -0.0766292	test: -0.0803866	best: -0.0803866 (82)	total: 508ms	remaining: 1.15s
83:	learn: -0.0730377	test: -0.0766871	best: -0.0766871 (83)	total: 515ms	remaining: 1.15s
84:	learn: -0.0695730	test: -0.0731283	best: -0.0731283 (84)	total: 520ms	remaining: 1.14s
85:	learn: -0.0663243	test: -0.0697712	best: -0.0697712 (85)	total: 531ms	remaining: 1.14s
86:	learn: -0.0630595	test: -0.0663741	best: -0.0663741 (86)	total: 534ms	remaining: 1.13s
87:	learn: -0.0600794	test: -0.0632876	best: -0.0632876 (87)	total: 536ms	remaining: 1.11s
88:	learn: -0.0572104	test: -0.0603181	best: -0.0603181 (88)	total: 540ms	remaining: 1.1s
89:	learn: -0.0543092	test: -0.0573246	best: -0.0573246 (89)	total: 546ms	remaining: 1.1s
90:	learn: -0.0515335	test: -0.0544631	best: -0.0544631 (90)	total: 553ms	remaining: 1.09s
91:	learn: -0.0489513	test: -0.0518015	best: -0.0518015 (91)	total: 560ms	remaining: 1.09s
92:	learn: -0.0464383	test: -0.0491886	best: -0.0491886 (92)	total: 566ms	remaining: 1.08s
9

185:	learn: 0.0210152	test: 0.0219152	best: 0.0219152 (185)	total: 913ms	remaining: 417ms
186:	learn: 0.0211151	test: 0.0220188	best: 0.0220188 (186)	total: 918ms	remaining: 412ms
187:	learn: 0.0212125	test: 0.0220938	best: 0.0220938 (187)	total: 922ms	remaining: 407ms
188:	learn: 0.0213306	test: 0.0222038	best: 0.0222038 (188)	total: 926ms	remaining: 402ms
189:	learn: 0.0214459	test: 0.0223552	best: 0.0223552 (189)	total: 930ms	remaining: 396ms
190:	learn: 0.0215429	test: 0.0224423	best: 0.0224423 (190)	total: 935ms	remaining: 391ms
191:	learn: 0.0216679	test: 0.0225908	best: 0.0225908 (191)	total: 941ms	remaining: 387ms
192:	learn: 0.0217663	test: 0.0227047	best: 0.0227047 (192)	total: 945ms	remaining: 382ms
193:	learn: 0.0218917	test: 0.0228714	best: 0.0228714 (193)	total: 948ms	remaining: 376ms
194:	learn: 0.0220129	test: 0.0230013	best: 0.0230013 (194)	total: 952ms	remaining: 371ms
195:	learn: 0.0221220	test: 0.0231356	best: 0.0231356 (195)	total: 955ms	remaining: 365ms
196:	learn

...

64:	learn: 0.0360926	test: 0.0286139	best: 0.0286139 (64)	total: 589ms	remaining: 1.68s
65:	learn: 0.0363863	test: 0.0287886	best: 0.0287886 (65)	total: 603ms	remaining: 1.69s
66:	learn: 0.0365208	test: 0.0287896	best: 0.0287896 (66)	total: 609ms	remaining: 1.67s
67:	learn: 0.0367731	test: 0.0288763	best: 0.0288763 (67)	total: 621ms	remaining: 1.67s
68:	learn: 0.0370888	test: 0.0289887	best: 0.0289887 (68)	total: 632ms	remaining: 1.67s
69:	learn: 0.0372635	test: 0.0290720	best: 0.0290720 (69)	total: 638ms	remaining: 1.65s
70:	learn: 0.0374777	test: 0.0292565	best: 0.0292565 (70)	total: 644ms	remaining: 1.63s
71:	learn: 0.0376574	test: 0.0293245	best: 0.0293245 (71)	total: 652ms	remaining: 1.62s
72:	learn: 0.0381128	test: 0.0293573	best: 0.0293573 (72)	total: 662ms	remaining: 1.61s
73:	learn: 0.0383411	test: 0.0293865	best: 0.0293865 (73)	total: 668ms	remaining: 1.6s
74:	learn: 0.0386182	test: 0.0295237	best: 0.0295237 (74)	total: 673ms	remaining: 1.58s
75:	learn: 0.0387490	test: 0.0296

167:	learn: 0.0631424	test: 0.0305551	best: 0.0313084 (125)	total: 1.42s	remaining: 701ms
168:	learn: 0.0634178	test: 0.0305345	best: 0.0313084 (125)	total: 1.43s	remaining: 696ms
169:	learn: 0.0635964	test: 0.0304946	best: 0.0313084 (125)	total: 1.45s	remaining: 691ms
170:	learn: 0.0636637	test: 0.0304937	best: 0.0313084 (125)	total: 1.46s	remaining: 683ms
171:	learn: 0.0638424	test: 0.0304387	best: 0.0313084 (125)	total: 1.47s	remaining: 673ms
172:	learn: 0.0640900	test: 0.0305891	best: 0.0313084 (125)	total: 1.47s	remaining: 664ms
173:	learn: 0.0646101	test: 0.0305813	best: 0.0313084 (125)	total: 1.48s	remaining: 656ms
174:	learn: 0.0650672	test: 0.0305474	best: 0.0313084 (125)	total: 1.49s	remaining: 647ms
175:	learn: 0.0654371	test: 0.0305217	best: 0.0313084 (125)	total: 1.5s	remaining: 637ms
176:	learn: 0.0655337	test: 0.0305252	best: 0.0313084 (125)	total: 1.5s	remaining: 628ms
177:	learn: 0.0657921	test: 0.0305363	best: 0.0313084 (125)	total: 1.51s	remaining: 618ms
178:	learn: 

17:	learn: -0.1070168	test: -0.1118465	best: -0.1118465 (17)	total: 149ms	remaining: 1.93s
18:	learn: -0.0888283	test: -0.0933395	best: -0.0933395 (18)	total: 160ms	remaining: 1.96s
19:	learn: -0.0731587	test: -0.0773324	best: -0.0773324 (19)	total: 173ms	remaining: 1.99s
20:	learn: -0.0598337	test: -0.0637668	best: -0.0637668 (20)	total: 182ms	remaining: 1.99s
21:	learn: -0.0474555	test: -0.0512442	best: -0.0512442 (21)	total: 193ms	remaining: 2.01s
22:	learn: -0.0374333	test: -0.0410475	best: -0.0410475 (22)	total: 201ms	remaining: 1.99s
23:	learn: -0.0288493	test: -0.0324023	best: -0.0324023 (23)	total: 210ms	remaining: 1.99s
24:	learn: -0.0211131	test: -0.0245404	best: -0.0245404 (24)	total: 216ms	remaining: 1.95s
25:	learn: -0.0143751	test: -0.0176378	best: -0.0176378 (25)	total: 222ms	remaining: 1.92s
26:	learn: -0.0088120	test: -0.0119195	best: -0.0119195 (26)	total: 230ms	remaining: 1.91s
27:	learn: -0.0039584	test: -0.0069666	best: -0.0069666 (27)	total: 239ms	remaining: 1.9s


130:	learn: 0.0495842	test: 0.0316498	best: 0.0316498 (130)	total: 1.17s	remaining: 1.07s
131:	learn: 0.0497082	test: 0.0315900	best: 0.0316498 (130)	total: 1.19s	remaining: 1.07s
132:	learn: 0.0500295	test: 0.0315920	best: 0.0316498 (130)	total: 1.2s	remaining: 1.06s
133:	learn: 0.0502979	test: 0.0316577	best: 0.0316577 (133)	total: 1.21s	remaining: 1.05s
134:	learn: 0.0505878	test: 0.0315952	best: 0.0316577 (133)	total: 1.22s	remaining: 1.04s
135:	learn: 0.0508796	test: 0.0315509	best: 0.0316577 (133)	total: 1.22s	remaining: 1.03s
136:	learn: 0.0511406	test: 0.0316728	best: 0.0316728 (136)	total: 1.23s	remaining: 1.03s
137:	learn: 0.0513864	test: 0.0317371	best: 0.0317371 (137)	total: 1.24s	remaining: 1.02s
138:	learn: 0.0516955	test: 0.0317955	best: 0.0317955 (138)	total: 1.25s	remaining: 1s
139:	learn: 0.0523329	test: 0.0318295	best: 0.0318295 (139)	total: 1.26s	remaining: 999ms
140:	learn: 0.0525437	test: 0.0319237	best: 0.0319237 (140)	total: 1.27s	remaining: 989ms
141:	learn: 0.

228:	learn: 0.0756379	test: 0.0312471	best: 0.0323618 (150)	total: 1.99s	remaining: 191ms
229:	learn: 0.0756870	test: 0.0312363	best: 0.0323618 (150)	total: 2s	remaining: 182ms
230:	learn: 0.0759366	test: 0.0312059	best: 0.0323618 (150)	total: 2.01s	remaining: 174ms
231:	learn: 0.0760599	test: 0.0311556	best: 0.0323618 (150)	total: 2.02s	remaining: 166ms
232:	learn: 0.0762450	test: 0.0311952	best: 0.0323618 (150)	total: 2.03s	remaining: 157ms
233:	learn: 0.0765854	test: 0.0312289	best: 0.0323618 (150)	total: 2.04s	remaining: 148ms
234:	learn: 0.0766882	test: 0.0311733	best: 0.0323618 (150)	total: 2.04s	remaining: 139ms
235:	learn: 0.0768501	test: 0.0311967	best: 0.0323618 (150)	total: 2.06s	remaining: 131ms
236:	learn: 0.0769722	test: 0.0312080	best: 0.0323618 (150)	total: 2.06s	remaining: 122ms
237:	learn: 0.0772106	test: 0.0310929	best: 0.0323618 (150)	total: 2.07s	remaining: 113ms
238:	learn: 0.0774562	test: 0.0311305	best: 0.0323618 (150)	total: 2.08s	remaining: 105ms
239:	learn: 0

## **Submission 양식 확인**

sample_submission.csv 화일 데이터(sample_submission)를 그대로 복사한 후, 
양식의 'ECLO' 컬럼에 test_x에 대한 ECLO(y) 예측값을 입력합니다 

In [18]:
catb_sub = sample_submission.copy()
catb_sub['ECLO'] = preds
catb_sub

,ID,ECLO
0,ACCIDENT_39609,4.443856
1,ACCIDENT_39610,3.910398
2,ACCIDENT_39611,6.111457
3,ACCIDENT_39612,5.321978
4,ACCIDENT_39613,5.188783
...,...,...
10958,ACCIDENT_50567,5.508619
10959,ACCIDENT_50568,4.678894
10960,ACCIDENT_50569,4.634458
10961,ACCIDENT_50570,4.702628


In [21]:
catb2_sub = sample_submission.copy()
catb2_sub['ECLO'] = preds2
catb2_sub

,ID,ECLO
0,ACCIDENT_39609,4.536890
1,ACCIDENT_39610,3.965983
2,ACCIDENT_39611,6.035167
3,ACCIDENT_39612,5.179383
4,ACCIDENT_39613,5.060512
...,...,...
10958,ACCIDENT_50567,5.949224
10959,ACCIDENT_50568,4.705312
10960,ACCIDENT_50569,4.855476
10961,ACCIDENT_50570,4.715671


## **답안지 저장 및 제출하기**

In [19]:
catb_sub.to_csv('catb_submit_split.csv', index=False)

In [22]:
catb2_sub.to_csv('catb2_submit.csv', index=False)